<a href="https://colab.research.google.com/github/alexander-toschev/ai-tools/blob/main/text/text_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Лекция: Обработка текстов в Python
Цель — познакомиться с основами обработки текстовых данных, включая очистку, векторизацию и классификацию.


## 📘 Что такое текстовые данные
- Неструктурированные: отзывы, чаты, документы
- Полу-структурированные: HTML, XML, JSON
- Структурированные: датафреймы с колонкой `text`


## 🛠 Библиотеки Python для NLP
- `nltk` — базовая обработка (токенизация, стемминг)
- `spaCy` — быстрый и мощный анализ: POS-теги, сущности
- `re` — регулярные выражения
- `scikit-learn` — векторизация текста (TF-IDF)
- `transformers` — предобученные модели BERT, GPT и др.

## 🔧 Очистка текста
- Приведение к нижнему регистру
- Удаление пунктуации, HTML-тегов
- Удаление стоп-слов
- Лемматизация и/или стемминг

## 📚 Что такое лемматизация?
**Лемматизация** — это процесс приведения слова к его базовой словарной форме.

**Примеры:**
- `running`, `ran`, `runs` → `run`
- `was`, `were` → `be`
- `better` → `good`

Цель — привести все формы одного слова к одной базе, чтобы улучшить анализ.


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("running"))        # ➜ run
print(lemmatizer.lemmatize("better", pos='a')) # ➜ good
print(lemmatizer.lemmatize("feet"))           # ➜ foot

### 📊 Лемматизация vs Стемминг
- **Стемминг** — обрезает окончания слов, быстрый, но грубый.
- **Лемматизация** — учитывает грамматику и использует словари.

|     | Стемминг | Лемматизация |
|-----|----------|---------------|
| Скорость | Быстрее | Медленнее |
| Точность | Ниже | Выше |
| Основа | Обрезка | Словари + грамматика |


## 🔠 Векторизация текста
- **Bag of Words**: частоты слов
- **TF-IDF**: частота с учетом важности
- **Word2Vec / BERT**: контекстные эмбеддинги


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = ["this is a text", "this is another text"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray())

[[0.         0.57735027 0.57735027 0.57735027]
 [0.63009934 0.44832087 0.44832087 0.44832087]]


## 🧠 Пример: Классификация текстов (20 newsgroups)
- Классы: спорт, медицина, компьютеры и т.д.
- Модель: `TfidfVectorizer` + `Naive Bayes`

In [5]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Load dataset
data = fetch_20newsgroups(subset='train', categories=['sci.med', 'rec.sport.baseball'])

# Create and train the model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(data.data, data.target)

# Predict category
text = "This is a bad example"
predicted_index = model.predict([text])[0]
predicted_category = data.target_names[predicted_index]

print(f"Text: '{text}'\nPredicted Category: {predicted_category}")

Text: 'This is a bad example'
Predicted Category: sci.med


## 📌 Вывод
- Текст нужно очищать и нормализовать
- Лемматизация помогает сократить словарь и повысить качество
- TF-IDF — хороший базовый способ векторизации
- Для сложных задач используют предобученные модели (BERT, GPT и др.)

## 🤖 Что такое BERT
**BERT (Bidirectional Encoder Representations from Transformers)** — это предобученная языковая модель от Google, которая понимает контекст слов в обе стороны — и слева, и справа.

Например, слово `bank` в двух разных предложениях:
- "He went to the **bank** to deposit money" → банк
- "She sat by the **bank** of the river" → берег

**BERT** понимает разницу и создает разные векторные представления.

Он используется для:
- Классификации текста
- Распознавания сущностей
- Вопрос-ответ систем
- Суммаризации

In [10]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
print(classifier('It more or less bad'))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9997377991676331}]


## 🔢 Сравнение методов векторизации текста


### 1. Bag of Words (BoW)
Превращает текст в вектор частот слов.
- Не учитывает порядок и контекст
- Прост в реализации

**Пример:**
```python
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(["I love NLP", "NLP is fun"])
print(X.toarray())
```

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(["I love NLP", "NLP is fun"])
print(X.toarray())

[[0 0 1 1]
 [1 1 0 1]]


### 2. TF-IDF (Term Frequency-Inverse Document Frequency)
Вектор, учитывающий частоту слова в документе и его редкость в коллекции.
- Снижает вес часто встречающихся слов (например, "the")
- Подходит для классификации

**Пример:**
```python
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(["this is a text", "this is another text"])
print(X.toarray())
```

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(["this is a text", "this is another text"])
print(X.toarray())

[[0.         0.57735027 0.57735027 0.57735027]
 [0.63009934 0.44832087 0.44832087 0.44832087]]


### 3. Word2Vec / GloVe
- Каждое слово представляется как плотный вектор (обычно длины 300)
- Похожее значение → похожий вектор
- Требует предварительного обучения или использования готовых весов

**Пример (gensim Word2Vec):**
```python
from gensim.models import Word2Vec
sentences = [["cat", "sits", "on", "the", "mat"]]
model = Word2Vec(sentences, vector_size=50, min_count=1)
print(model.wv['cat'])
```

In [21]:
!pip install gensim==4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (gensim)


In [15]:
from gensim.models import Word2Vec
sentences = [["cat", "sits", "on", "the", "mat"]]
model = Word2Vec(sentences, vector_size=50, min_count=1)
print(model.wv['cat'])

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [18]:
import gensim
print(gensim.__version__)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

### 4. BERT / Sentence-BERT
- Векторизует слова или целые предложения
- Учитывает порядок, контекст, грамматику
- Универсален для любых NLP задач

**Пример:**
```python
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding = model.encode("Natural Language Processing is cool")
print(embedding.shape)  # ➜ (384,)
```

In [17]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding = model.encode("Natural Language Processing is cool")
print(embedding.shape)  # ➜ (384,)
print(embedding)

(384,)
[-1.89818591e-02 -5.96635938e-02  4.80235033e-02 -2.05745585e-02
  2.42722854e-02 -7.97964782e-02  2.66040955e-02 -1.70274898e-02
  4.15130518e-02  5.52435070e-02  4.61376682e-02 -3.16511118e-03
  4.52745333e-02  1.72013566e-02  1.01983259e-02  7.87022486e-02
  2.16608848e-02  5.29048778e-02 -1.02077059e-01 -1.07588083e-01
 -6.50576726e-02  7.96919093e-02 -4.26407158e-02 -5.94089068e-02
  2.59572975e-02  7.81594664e-02 -5.58298230e-02 -5.00910804e-02
  9.91702899e-02 -4.15725354e-03 -1.29467361e-02  6.03717156e-02
  2.21197605e-02  8.94275829e-02 -2.30027288e-02 -3.41766328e-02
  2.61088517e-02  5.54244742e-02 -2.12720726e-02  3.26759042e-03
 -9.39921588e-02 -2.69461628e-02  3.32310959e-03  4.28451551e-03
  1.24104291e-01  3.18499319e-02 -7.43955970e-02 -1.36972603e-03
  3.79725471e-02  3.21679078e-02 -9.83800068e-02 -4.29900326e-02
  7.89796526e-04  1.10257976e-01 -7.02990443e-02  8.25974643e-02
 -4.00876366e-02 -6.72073886e-02 -5.30215502e-02 -2.86133550e-02
 -2.99126990e-02 -

### 📊 Сравнение методов
| Метод      | Учитывает порядок? | Контекст? | Тип выхода | Размерность |
|------------|--------------------|-----------|------------|-------------|
| BoW        | ❌                 | ❌        | Документ   | #слов       |
| TF-IDF     | ❌                 | ❌        | Документ   | #слов       |
| Word2Vec   | ❌                 | ✅ частично | Слово      | ~300        |
| BERT       | ✅                 | ✅        | Слово/фраза| 768+        |